## ABSA with ATE —> DeBERTa pipeline

In [ ]:
%pip install spacy transformers

In [ ]:
import spacy
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Spacy model
nlp = spacy.load("en_core_web_sm")

# Pre-trained DeBERTa model for absa
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


/Users/yusuf/dev/models/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/yusuf/dev/models/.venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [2]:
def extract_aspects(text):
    doc = nlp(text)
    aspects = []
    for chunk in doc.noun_chunks:  # extract noun phrases
        if any(token.dep_ in ("nsubj", "dobj") for token in chunk):  # focus on key aspects
            aspects.append(chunk.text)  # store the full noun phrase
    return aspects

In [ ]:
def analyze_aspect_sentiment(text, aspects):
    sentiment_results = {}
    for aspect in aspects:
        result = classifier(text, text_pair=aspect)[0]  # get full result dict
        sentiment = result['label']
        confidence = round(result['score'], 5)  # round to 5 digits
        sentiment_results[aspect] = {'sentiment': sentiment, 'confidence': confidence}
    return sentiment_results

In [8]:
# example text
text = "The camera quality of this phone is amazing, but the battery life is disappointing."

# extract aspects
aspects = extract_aspects(text)
print("Extracted Aspects:", aspects)

# get sentiment for each aspect
aspect_sentiments = analyze_aspect_sentiment(text, aspects)
print("Aspect Sentiment Analysis:", aspect_sentiments)

Extracted Aspects: ['The camera quality', 'the battery life']
Aspect Sentiment Analysis: {'The camera quality': {'sentiment': 'Positive', 'confidence': 0.99813}, 'the battery life': {'sentiment': 'Negative', 'confidence': 0.99647}}
